Build Tree Model

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

Load input

In [7]:
df_fips=pd.read_csv('../../input/bees/targets/bees-targets_new.csv')

print(df_fips)

       Unnamed: 0       Name    Fips      State  State ANSI   Ag District  \
0               0  aleutians  2013.0        NaN         NaN           NaN   
1               1  aleutians  2016.0        NaN         NaN           NaN   
2               2  anchorage  2020.0        NaN         NaN           NaN   
3               3     bethel  2050.0        NaN         NaN           NaN   
4               4    bristol  2060.0        NaN         NaN           NaN   
...           ...        ...     ...        ...         ...           ...   
14696       14696        NaN     NaN  WISCONSIN        55.0       CENTRAL   
14697       14697        NaN     NaN  WISCONSIN        55.0  EAST CENTRAL   
14698       14698        NaN     NaN  WISCONSIN        55.0  WEST CENTRAL   
14699       14699        NaN     NaN  WISCONSIN        55.0  WEST CENTRAL   
14700       14700        NaN     NaN    WYOMING        56.0     NORTHWEST   

       Ag District Code       County  County ANSI   2002  ...   2022     FI

In [8]:
# Check available states and years in the dataset
print("Available states:", df_fips['State'].unique())
print("Available years:", [2002, 2007, 2012, 2017])

Available states: [nan 'ALABAMA' 'GEORGIA' 'WEST VIRGINIA' 'TENNESSEE' 'IOWA' 'KANSAS'
 'KENTUCKY' 'MISSOURI' 'NEBRASKA' 'OHIO' 'PENNSYLVANIA' 'ARKANSAS'
 'FLORIDA' 'ILLINOIS' 'MICHIGAN' 'MISSISSIPPI' 'SOUTH CAROLINA' 'TEXAS'
 'OKLAHOMA' 'NORTH CAROLINA' 'VIRGINIA' 'INDIANA' 'MINNESOTA'
 'SOUTH DAKOTA' 'IDAHO' 'LOUISIANA' 'MAINE' 'MASSACHUSETTS' 'NEW YORK'
 'VERMONT' 'WASHINGTON' 'COLORADO' 'OREGON' 'WISCONSIN' 'MONTANA'
 'MARYLAND' 'UTAH' 'RHODE ISLAND' 'NEW HAMPSHIRE' 'NEVADA' 'NEW MEXICO'
 'NORTH DAKOTA' 'WYOMING' 'CALIFORNIA' 'NEW JERSEY' 'ARIZONA' 'DELAWARE'
 'HAWAII' 'CONNECTICUT']
Available years: [2002, 2007, 2012, 2017]


In [17]:
# Prompt- user for input
selected_state = input("Enter the state (e.g., MAINE): ").strip()

selected_year = int(input("Enter the year (2002, 2007, 2012, 2017, or 2022): ").strip())

# Filter the DataFrame based on the user input
density_col = f'{selected_year}_bee_density'

# Filter data
df_filtered = df_fips[(df_fips['State'] == selected_state) & (df_fips[density_col].notna())].copy()

# Create label based on top 20% bee density
threshold = df_filtered[density_col].quantile(0.8)
df_filtered['label'] = (df_filtered[density_col] > threshold).astype(int)

# Display the filtered data
print(f"Filtered data for {selected_state} in {selected_year}:")
print(df_filtered.head())

Enter the state (e.g., MAINE): MAINE
Enter the year (2002, 2007, 2012, 2017, or 2022): 2007
Filtered data for MAINE in 2007:
     Unnamed: 0        Name    Fips  State  State ANSI Ag District  \
141         141    franklin  1059.0  MAINE        23.0       SOUTH   
437         437  washington  1129.0  MAINE        23.0     CENTRAL   
596         596    franklin  5047.0  MAINE        23.0       SOUTH   
748         748     lincoln  5079.0  MAINE        23.0       SOUTH   
976         976  washington  5143.0  MAINE        23.0     CENTRAL   

     Ag District Code      County  County ANSI   2002  ...    FIPS  \
141              30.0    franklin          7.0   78.0  ...  1059.0   
437              20.0  washington         29.0  346.0  ...  1129.0   
596              30.0    franklin          7.0   78.0  ...  5047.0   
748              30.0     lincoln         15.0  517.0  ...  5079.0   
976              20.0  washington         29.0  346.0  ...  5143.0   

         county     state  Total_

In [18]:
df_filtered.head()

,Unnamed: 0,Name,Fips,State,State ANSI,Ag District,Ag District Code,County,County ANSI,2002,...,FIPS,county,state,Total_Area_km2,2002_bee_density,2007_bee_density,2012_bee_density,2017_bee_density,2022_bee_density,label
141,141,franklin,1059.0,MAINE,23.0,SOUTH,30.0,franklin,7.0,78.0,...,1059.0,franklin,alabama,1674.485326,0.046581,0.042998,0.097941,0.038818,0.043595,0
437,437,washington,1129.0,MAINE,23.0,CENTRAL,20.0,washington,29.0,346.0,...,1129.0,washington,alabama,2819.372979,0.122722,0.258568,2.494172,0.168832,0.033695,0
596,596,franklin,5047.0,MAINE,23.0,SOUTH,30.0,franklin,7.0,78.0,...,5047.0,franklin,arkansas,1606.071959,0.048566,0.044830,0.102112,0.040471,0.045453,0
748,748,lincoln,5079.0,MAINE,23.0,SOUTH,30.0,lincoln,15.0,517.0,...,5079.0,lincoln,arkansas,1475.702552,0.350342,0.022362,0.029139,0.136206,0.174832,0
976,976,washington,5143.0,MAINE,23.0,CENTRAL,20.0,washington,29.0,346.0,...,5143.0,washington,arkansas,2466.244494,0.140294,0.295591,2.851299,0.193006,0.038520,0


In [19]:
label_counts = df_filtered['label'].value_counts()
print(label_counts)

label
0    97
1    24
Name: count, dtype: int64


Procedure to run a single state
Take Maine as an example

In [20]:
#ML MODEL-1ST HALF (ONE HOT ENCODING)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score


# Define Features and Target with One-Hot Encoding
#df_full_model = df_fips[df_fips['State'] == 'MAINE']  # Filter for Maine
categorical_columns = ['County', 'State', 'Ag District'] # Categorical columns for one-hot encoding

# Apply one-hot encoding
df_encoded = pd.get_dummies(df_filtered, columns=categorical_columns, drop_first=True)
df_encoded.columns
df_filtered.columns


Index(['Unnamed: 0', 'Name', 'Fips', 'State', 'State ANSI', 'Ag District',
       'Ag District Code', 'County', 'County ANSI', '2002', '2007', '2012',
       '2017', '2022', 'FIPS', 'county', 'state', 'Total_Area_km2',
       '2002_bee_density', '2007_bee_density', '2012_bee_density',
       '2017_bee_density', '2022_bee_density', 'label'],
      dtype='object')

In [23]:
#MODEL-2ND HALF (TRAINING AND EVALUATION)

# Define features (X) and target (y)
X = df_encoded.drop(columns=['Unnamed: 0','Name','State ANSI',
       'Ag District Code', 'County ANSI', '2002', '2007', '2012',
       '2017', '2022', 'FIPS', 'county', 'state', 'Total_Area_km2',
       '2002_bee_density', '2007_bee_density', '2012_bee_density',
       '2017_bee_density', 'label'])  # Drop non-feature columns
y = df_encoded['label']  # Target variable

X.columns
# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Perform Grid Search for Random Forest Hyperparameters
rfc = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 25],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [5, 10]
}
rfcs = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
rfcs.fit(X_train, y_train)

# Save optimal parameters
n_estimator = rfcs.best_params_['n_estimators']
depth = rfcs.best_params_['max_depth']
min_split = rfcs.best_params_['min_samples_split']
min_leaf = rfcs.best_params_['min_samples_leaf']
print(f'n_estimators = {n_estimator}, max_depth = {depth}, min_samples_split = {min_split}, min_samples_leaf = {min_leaf}')

# Fit a new Random Forest with optimal parameters
opt_rfc = RandomForestClassifier(
    n_estimators=n_estimator,
    max_depth=depth,
    min_samples_split=min_split,
    min_samples_leaf=min_leaf,
    random_state=42,
    class_weight='balanced'
)
opt_rfc.fit(X_train, y_train)
pred_train = opt_rfc.predict(X_train)
pred_test = opt_rfc.predict(X_test)

# Evaluate Model
accu_train = accuracy_score(y_train, pred_train)
accu_test = accuracy_score(y_test, pred_test)

f1_train = f1_score(y_train, pred_train)
f1_test = f1_score(y_test, pred_test)

recall_train = recall_score(y_train, pred_train)
recall_test = recall_score(y_test, pred_test)

precision_train = precision_score(y_train, pred_train)
precision_test = precision_score(y_test, pred_test)

# Print Results
print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')


n_estimators = 50, max_depth = 10, min_samples_split = 10, min_samples_leaf = 5
Accuracy score on train set = 0.9583333333333334 and accuracy score on test set = 0.92
f1 score on train set = 0.8823529411764706 and f1 score on test set = 0.75
Recall score on train set = 0.7894736842105263 and recall score on test set = 0.6
Precision score on train set = 1.0 and precision score on test set = 1.0


In [ ]:
states = ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
          "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
          "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
          "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
          "WV", "WY"]

In [ ]:
# #improved version-2
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# # Load Data
# ##df = pd.read_csv("https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/targets/bees-targets.csv")
# #df_fips = pd.read_csv('../../input/bees/targets/bees-targets.csv')
# #df_fips['label'] = df_fips['2022_increase']

# # Define Features and Target with One-Hot Encoding
# # = df_fips[df_fips['State'] == 'MAINE']  # Filter for Maine
# categorical_columns = ['County', 'State', 'Ag District']  # Categorical columns for one-hot encoding

# # Apply one-hot encoding
# df_encoded = pd.get_dummies(df_full_model, columns=categorical_columns, drop_first=True)

# # Define features (X) and target (y)
# X = df_encoded.drop(columns=['Fips', 'Name', 'label'])  # Drop non-feature columns
# y = df_encoded['label']  # Target variable

# # Split Dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# # Standardize Features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Perform Grid Search for Random Forest Hyperparameters
# rfc = RandomForestClassifier()
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [5, 10, 20],
#     'min_samples_split': [10, 20],
#     'min_samples_leaf': [5, 10]
# }
# rfcs = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
# rfcs.fit(X_train, y_train)

# # Save optimal parameters
# n_estimator = rfcs.best_params_['n_estimators']
# depth = rfcs.best_params_['max_depth']
# min_split = rfcs.best_params_['min_samples_split']
# min_leaf = rfcs.best_params_['min_samples_leaf']
# print(f'n_estimators = {n_estimator}, max_depth = {depth}, min_samples_split = {min_split}, min_samples_leaf = {min_leaf}')

# # Fit a new Random Forest with optimal parameters
# opt_rfc = RandomForestClassifier(
#     n_estimators=n_estimator,
#     max_depth=depth,
#     min_samples_split=min_split,
#     min_samples_leaf=min_leaf,
#     random_state=42
# )
# opt_rfc.fit(X_train, y_train)
# pred_train = opt_rfc.predict(X_train)
# pred_test = opt_rfc.predict(X_test)

# # Evaluate Model
# accu_train = accuracy_score(y_train, pred_train)
# accu_test = accuracy_score(y_test, pred_test)

# f1_train = f1_score(y_train, pred_train)
# f1_test = f1_score(y_test, pred_test)

# recall_train = recall_score(y_train, pred_train)
# recall_test = recall_score(y_test, pred_test)

# precision_train = precision_score(y_train, pred_train)
# precision_test = precision_score(y_test, pred_test)

# # Print Results
# print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
# print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
# print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
# print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')


In [ ]:
# df_full_model=df_fips[df_fips['State'] == 'MAINE']



# #X = df_full_model.drop(columns=['Fips', 'State','county', 'Ag District','Program',''])
# X=df_full_model[['FIPS','county','State','Total_Area_km2']]

# y = df_full_model['label']


# #X.head()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# rfc=RandomForestClassifier()
# param_grid = {
#     'n_estimators': [50, 100, 150, 200, 250],
#     'max_depth' : [5, 10, 20, 50, 80]
# }
# rfcs = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# rfcs.fit(X_train, y_train)
# #save optimal parameters
# n_estimator = rfcs.best_params_['n_estimators']
# depth = rfcs.best_params_['max_depth']
# print(f'n_estimators = {n_estimator} and max_depth = {depth}')

# #fit a new random forest with optimal parameters
# opt_rfc = RandomForestClassifier(n_estimators= n_estimator, max_depth=depth)
# opt_rfc.fit(X_train, y_train)
# pred_train = opt_rfc.predict(X_train)
# pred_test = opt_rfc.predict(X_test)

# accu_train = accuracy_score(y_train, pred_train)
# accu_test = accuracy_score(y_test, pred_test)

# f1_train = f1_score(y_train, pred_train)
# f1_test = f1_score(y_test, pred_test)

# recall_train = recall_score(y_train, pred_train)
# recall_test = recall_score(y_test, pred_test)

# precision_train = precision_score(y_train, pred_train)
# precision_test = precision_score(y_test, pred_test)
# print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
# print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
# print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
# print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')

In [ ]:
# #one hot encoding
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# # Load Data
# df = pd.read_csv("https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/targets/bees-targets.csv")
# df_fips = pd.read_csv('../../input/bees/targets/bees-targets.csv')
# df_fips['label'] = df_fips['2022_increase']

# # Define Features and Target with One-Hot Encoding
# df_full_model = df_fips[df_fips['State'] == 'MAINE']  # Filter for Maine
# categorical_columns = ['County', 'State', 'Ag District']  # Categorical columns for one-hot encoding

# # Apply one-hot encoding
# df_encoded = pd.get_dummies(df_full_model, columns=categorical_columns, drop_first=True)

# # Define features (X) and target (y)
# X = df_encoded.drop(columns=['Fips', 'Name', 'label'])  # Drop non-feature columns
# y = df_encoded['label']  # Target variable

# # Split Dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# # Standardize Features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Perform Grid Search for Random Forest Hyperparameters
# rfc = RandomForestClassifier()
# param_grid = {
#     'n_estimators': [50, 100, 150, 200, 250],
#     'max_depth': [5, 10, 20, 50, 80]
# }
# rfcs = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
# rfcs.fit(X_train, y_train)

# # Save optimal parameters
# n_estimator = rfcs.best_params_['n_estimators']
# depth = rfcs.best_params_['max_depth']
# print(f'n_estimators = {n_estimator} and max_depth = {depth}')

# # Fit a new Random Forest with optimal parameters
# opt_rfc = RandomForestClassifier(n_estimators=n_estimator, max_depth=depth)
# opt_rfc.fit(X_train, y_train)
# pred_train = opt_rfc.predict(X_train)
# pred_test = opt_rfc.predict(X_test)

# # Evaluate Model
# accu_train = accuracy_score(y_train, pred_train)
# accu_test = accuracy_score(y_test, pred_test)

# f1_train = f1_score(y_train, pred_train)
# f1_test = f1_score(y_test, pred_test)

# recall_train = recall_score(y_train, pred_train)
# recall_test = recall_score(y_test, pred_test)

# precision_train = precision_score(y_train, pred_train)
# precision_test = precision_score(y_test, pred_test)

# print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
# print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
# print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
# print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')
